In [1]:
import keras
from keras.models import Sequential
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization, Input
from keras.layers import Conv2D, MaxPool2D, Add, Lambda, Concatenate, Reshape
from keras.models import Model
from keras.datasets import cifar10
from keras import regularizers
from keras.callbacks import LearningRateScheduler
import tensorflow as tf
import numpy as np
import weave
import json
import time

/Users/Tim/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
keras.backend.image_data_format()

'channels_first'

In [3]:
batch_size = 64
num_classes = 10
steps_per_epoch = 781
epochs = 1
num_val_test = 10000

In [4]:
def lr_schedule(epoch):
    return (0.00100 - 7e-6*epoch)

class TimeHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.times = []

    def on_epoch_begin(self, batch, logs={}):
        self.epoch_time_start = time.time()

    def on_epoch_end(self, batch, logs={}):
        self.times.append(time.time() - self.epoch_time_start)
        
viewer = keras.callbacks.TensorBoard(log_dir='./logs', 
                                     histogram_freq=0, 
                                     batch_size=batch_size, 
                                     write_graph=True)


In [9]:
def unstack_shape(input_shape):
    return [(input_shape[0], input_shape[2], input_shape[3])]*input_shape[1]

def keras_unstack(x):
    return tf.unstack(x, axis = 1)

def half_Conv(x, num_filters):
    new_shape = x.shape.as_list()
    
    x = Lambda(keras_unstack, output_shape=unstack_shape)(x)
    xA = x[:len(x)//2]
    xB = x[len(x)//2:]
    xA_shape = []
    xB_shape = []
    for n in xA:
        xA_shape.append(Reshape((1, new_shape[2], new_shape[3]))(n))
    for n in xB:
        xB_shape.append(Reshape((1, new_shape[2], new_shape[3]))(n))
    print(len(xA_shape), len(xB_shape))
    xA = Concatenate(axis = 1)(xA_shape)
    xB = Concatenate(axis = 1)(xB_shape)
    
    x = weave.pyrm_weave_disjoint_pure_combine([xA,xB], num_filters,
                                               devices=["/cpu:0","/cpu:0"])
    return x

In [14]:
def half_straight_stack(x, num_filters, rep = 3, drop = 0.2):
    x = Conv2D(num_filters, (3,3), padding='same', activation='relu')(x)
    for n in range(rep):
        if (n != 0):
            x = half_Conv(x, (2**n)*num_filters)
        x = BatchNormalization()(x)
        x = half_Conv(x, (2**n)*num_filters)
        x = BatchNormalization()(x)
        x = MaxPool2D()(x)
        x = Dropout(drop)(x)
    
    return x

def straight_stack(x, num_filters, rep = 3, drop = 0.2):
    for n in range(rep):
        x = Conv2D((2**n)*num_filters, (3,3), padding='same', activation='relu')(x)
        x = BatchNormalization()(x)
        x = Conv2D((2**n)*num_filters, (3,3), padding='same', activation='relu')(x)
        x = BatchNormalization()(x)
        x = MaxPool2D()(x)
        x = Dropout(drop)(x)
    
    return x

In [15]:
inputs = Input(shape=(3,32,32))
x = straight_stack(inputs, 24)
x = Flatten()(x)

predictions = Dense(num_classes, activation='softmax')(x)
model = Model(inputs=[inputs], outputs=predictions)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

inputs = Input(shape=(3,32,32))
x = half_straight_stack(inputs, 32)
x = Flatten()(x)

predictions = Dense(num_classes, activation='softmax')(x)
model2 = Model(inputs=[inputs], outputs=predictions)
model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model2.summary()




_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         (None, 3, 32, 32)         0         
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 24, 32, 32)        672       
_________________________________________________________________
batch_normalization_19 (Batc (None, 24, 32, 32)        128       
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 24, 32, 32)        5208      
_________________________________________________________________
batch_normalization_20 (Batc (None, 24, 32, 32)        128       
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 24, 16, 16)        0         
_________________________________________________________________
dropout_10 (Dropout)         (None, 24, 16, 16)        0         
__________

In [16]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

print(x_train.shape)
 
#z-score
mean = np.mean(x_train,axis=(0,1,2,3))
std = np.std(x_train,axis=(0,1,2,3))
x_train = (x_train-mean)/(std+1e-7)
x_test = (x_test-mean)/(std+1e-7)
 
y_train = np_utils.to_categorical(y_train,num_classes)
y_test = np_utils.to_categorical(y_test,num_classes)

datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    )
datagen.fit(x_train)

(50000, 3, 32, 32)


In [17]:
model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                    steps_per_epoch= steps_per_epoch,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test[0:num_val_test],y_test[0:num_val_test]),
                    callbacks=[LearningRateScheduler(lr_schedule)]).history

Epoch 1/1
781/781 [==============================] - 1142s 1s/step - loss: 1.6067 - acc: 0.4343 - val_loss: 1.1326 - val_acc: 0.5888


{'acc': [0.43445610381288047],
 'loss': [1.6067857215970573],
 'val_acc': [0.5888],
 'val_loss': [1.132638637161255]}

In [18]:
model2.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                    steps_per_epoch= steps_per_epoch,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test[0:num_val_test],y_test[0:num_val_test]),
                    callbacks=[LearningRateScheduler(lr_schedule)]).history

Epoch 1/1
781/781 [==============================] - 3224s 4s/step - loss: 1.7811 - acc: 0.3927 - val_loss: 1.3421 - val_acc: 0.5070


{'acc': [0.39292294136494715],
 'loss': [1.7801973117692698],
 'val_acc': [0.507],
 'val_loss': [1.3420563457489014]}

In [19]:
model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                    steps_per_epoch= steps_per_epoch,
                    epochs=2,
                    verbose=1,
                    validation_data=(x_test[0:num_val_test],y_test[0:num_val_test]),
                    callbacks=[LearningRateScheduler(lr_schedule)]).history

Epoch 1/2
781/781 [==============================] - 1077s 1s/step - loss: 1.1351 - acc: 0.5954 - val_loss: 1.0005 - val_acc: 0.6576
Epoch 2/2
781/781 [==============================] - 1079s 1s/step - loss: 0.9627 - acc: 0.6600 - val_loss: 0.8286 - val_acc: 0.7206


{'acc': [0.5954021147068247, 0.6600048061518744],
 'loss': [1.1349386094052683, 0.962666728602736],
 'val_acc': [0.6576, 0.7206],
 'val_loss': [1.0004543922424316, 0.8286045732498168]}

In [20]:
model2.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                    steps_per_epoch= steps_per_epoch,
                    epochs=2,
                    verbose=1,
                    validation_data=(x_test[0:num_val_test],y_test[0:num_val_test]),
                    callbacks=[LearningRateScheduler(lr_schedule)]).history

Epoch 1/2
781/781 [==============================] - 3226s 4s/step - loss: 1.3378 - acc: 0.5159 - val_loss: 1.2074 - val_acc: 0.5653
Epoch 2/2
781/781 [==============================] - 3182s 4s/step - loss: 1.1918 - acc: 0.5716 - val_loss: 1.1697 - val_acc: 0.5962


{'acc': [0.5159804549823774, 0.5716717398269785],
 'loss': [1.3377229910330755, 1.1919392397372397],
 'val_acc': [0.5653, 0.5962],
 'val_loss': [1.207352855682373, 1.1697087383270264]}

In [ ]:
#Method to decrease parameter counts! 
#x = Conv2D(2*Y)(x)
#size(x) = (?, 2*Y, H, W)
#[x1, x2] = unstack(x)
#size(x1) = (?, Y, H, W)
#x = weave(x1,x2)
#size(x) = (?, Y, H, W) (Did we reduce parmaeters and actaully keep more than half of the data)

In [ ]:
#FC network of convoltuon Filters:
#Layer 0 = input (?, 3, 32, 32):
#1 = N Conv2d(1)(layer 0)
#Change number of convolution layers to number of GPUS.
#Then split up the output to seperate sections. This will effectively act as many convolution layers in parallel.

#Set number of hidden layers in each group as a New GPU node.
